In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install bitsandbytes # 모델을 경량화해서 로드
!pip install transformers
!pip install peft # LoRA 튜닝을 하게 해주는 라이브러리
!pip install accelerate
!pip install trl
!pip install datasets

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from trl import SFTTrainer
import os
import re
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### preprocess

In [ ]:
# train = pd.read_csv('../train_preprocess_data_aug.csv')
train = pd.read_csv('/content/drive/MyDrive/데이콘/train_preprocess_data_aug.csv')
train

,질문,답변
0,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,면진장치가 뭐야?,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을..."
2,면진장치가 뭐야?,면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...
3,면진장치가 뭐야?,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...
4,면진장치가 뭐야?,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
...,...,...
6997,현관문 주위 결로의 대책은 뭐야? 또한 현관문 주위 결로의 원인은 뭐야??,현관문 주위 결로의 대책은 규격제품을 사용하고 방화문 현관문 내부 종이 허니컴 사이...
6998,"부엌 및 욕실의 결로의 원인은 뭐야? 또한 AD, PD에 면한 벽체 결로에 대한 대...",부엌이나 욕실의 경우 외벽이나 비 난방공간에 접한 면이 벽체의 일부인 부엌이나 욕실...
6999,발코니에 결로가 발생하는 원인은 뭐야? 또한 벽장 부위 결로의 원인이 뭐야??,발코니에 결로가 발생하는 원인은 발코니에 단열 및 방습조치가 되지 않은 상태에서 실...
7000,부엌 및 욕실의 결로의 대책은 뭐야? 또한 외벽 모서리 부위에 결로가 발생하는 원인...,부엌 및 욕실의 결로에 대한 대책은 환기구 설치입니다. 부엌 및 욕실의 결로는 주로...


### train

In [ ]:
# output_dir = "llama2_13b_finetuned_preprocess_data_aug"
output_dir = "/content/drive/MyDrive/데이콘/llama-2-koen-13b/llama2_13b_finetuned_preprocess_data_aug"
model_id = "beomi/llama-2-koen-13b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, eos_token="<|endoftext|>")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}) # 모델을 쪼개서 load함

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(46336, 5120, padding_idx=2)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    

In [ ]:
tokenizer

LlamaTokenizerFast(name_or_path='beomi/llama-2-koen-13b', vocab_size=46336, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|endoftext|>', 'unk_token': '<unk>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46331: AddedToken("<|sep|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46332: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46333: AddedToken("<|acc|>", rstrip=False, lstrip=False, single_word=False, normalized=False, spe

In [ ]:
# "당신은 질문에 답변하는 역할을 하는 챗봇입니다. 사용자의 질문에 올바른 답변을 하세요.\n### 질문: {x['question']}\n### 답변: {x['answer']}<|endoftext|>"

In [ ]:
dataset = []
for question, answer in tqdm(zip(train['질문'], train['답변']), total = len(train)):
    dataset.append(f"당신은 질문에 답변하는 역할을 하는 챗봇입니다. 사용자의 질문에 올바른 답변을 하세요.\n### 질문: {question}\n### 답변: {answer}"+tokenizer.eos_token)
dataset = Dataset.from_dict({"text": dataset})
dataset = dataset.shuffle()
dataset

  0%|          | 0/7002 [00:00<?, ?it/s]

Dataset({
    features: ['text'],
    num_rows: 7002
})

In [ ]:
dataset['text'][0]

'당신은 질문에 답변하는 역할을 하는 챗봇입니다. 사용자의 질문에 올바른 답변을 하세요.\n### 질문: 유럽의 클래식한 디자인을 집 내부에 반영하는 데 어떤 포인트를 주의해야 할까요?\n### 답변: 유럽 클래식한 디자인을 집 내부에 반영하려면 몇 가지 주의해야 할 점이 있습니다. 먼저, 디테일을 중요시하는 유럽 클래식한 스타일을 채택하는 것이 중요합니다. 대리석 가구와 같은 고급스러운 소재와 장식품을 사용하고, 서스펜션 조명을 선택하여 전통적이면서도 우아한 분위기를 연출할 수 있습니다. 또한, 정교한 아키텍처 디테일을 강조하여 집 내부에 유럽 클래식한 분위기를 조성할 수 있습니다. 따라서 건축 및 가구 디자인에 신중한 선택이 필요합니다.<|endoftext|>'

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
lora_config_dict = {
    'lora_r' : 8,
    'lora_alpha' : 32,
    'lora_dropout' : 0.05,
    'lora_bias' : "none",
    'lora_task_type' : "CAUSAL_LM",
    'target_modules': ["q_proj", "up_proj", "o_proj", "k_proj", "down_proj", "gate_proj", "v_proj"]
}

peft_config = LoraConfig(
            lora_alpha=lora_config_dict['lora_alpha'],
            lora_dropout=lora_config_dict['lora_dropout'],
            r=lora_config_dict['lora_r'],
            bias=lora_config_dict['lora_bias'],
            task_type=lora_config_dict['lora_task_type'],
            target_modules = lora_config_dict['target_modules']
        )

In [ ]:
train_param = {
    'num_train_epochs': 6,
    'per_device_train_batch_size': 4,
    'per_device_eval_batch_size': 4,
    'gradient_accumulation_steps': 1,
    'optim': "paged_adamw_8bit",
    'save_steps': 500,
    'logging_steps': 10,
    'learning_rate': 0.0002,
    'weight_decay':  0.01,
    'max_grad_norm': 1,
    'warmup_ratio': 0.06,
    'group_by_length' : False,
    'lr_scheduler_type': 'cosine'
}

training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=train_param['num_train_epochs'],
            per_device_train_batch_size=train_param['per_device_train_batch_size'],
            per_device_eval_batch_size=train_param['per_device_eval_batch_size'],
            gradient_accumulation_steps=train_param['gradient_accumulation_steps'],
            optim=train_param['optim'],
            save_steps=train_param['save_steps'],
            logging_steps=train_param['logging_steps'],
            learning_rate=train_param['learning_rate'],
            weight_decay=train_param['weight_decay'],
            # fp16=kwargs['fp16'],
            # bf16=kwargs['bf16'],
            max_grad_norm=train_param['max_grad_norm'],
            # max_steps=kwargs['max_steps'],
            warmup_ratio=train_param['warmup_ratio'],
            group_by_length=train_param['group_by_length'],
            lr_scheduler_type=train_param['lr_scheduler_type'],
            seed=42,
#             report_to="tensorboard"
        )

In [ ]:
trainer = SFTTrainer(
            model=model,
            train_dataset=dataset,
            peft_config=peft_config,
            dataset_text_field="text",
            max_seq_length=None,
            tokenizer=tokenizer,
            args=training_args,
            packing=False,
            # seed=42,
            data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
            # neftune_noise_alpha=5,
            # callbacks=[AbortCallback()]
            )

Map:   0%|          | 0/7002 [00:00<?, ? examples/s]

In [ ]:
model.config.use_cache = False
# trainer.train(resume_from_checkpoint=False)
trainer.train(resume_from_checkpoint=True) # checkpoint에 이어서 학습할 때 사용

Step,Training Loss
8510,0.084700
8520,0.083700
8530,0.086900
8540,0.089500
8550,0.097700
8560,0.087100
8570,0.083000
8580,0.091700
8590,0.084300
8600,0.087400


TrainOutput(global_step=10506, training_loss=0.0167406683714304, metrics={'train_runtime': 8068.6155, 'train_samples_per_second': 5.207, 'train_steps_per_second': 1.302, 'total_flos': 5.920877586933965e+17, 'train_loss': 0.0167406683714304, 'epoch': 6.0})

In [ ]:
trainer.model.save_pretrained(output_dir)